In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
train = pd.read_pickle(FILE.train_ori.value)
test = pd.read_pickle(FILE.test_ori.value)
X = pd.concat([train,test],sort=False)

# tag sequence

In [31]:
def tag(x,length=50):
    split = x.split(',')
    pad = [np.nan for i in range(length-len(split))]
    split.extend(pad)
    return npsplit[:length]

returns = X['user_tags'].astype(str).apply(tag,length=50)
returns = np.array(list(returns))
df = pd.DataFrame(data=returns,columns=['tag_{}'.format(i) for i in range(50)])
df.head()
    
le = LabelEncoder()
for col in tqdm(df.columns):
    df[col] = le.fit_transform(df[col].astype(str))
df['instance_id'] = X['instance_id'].values
df.to_pickle('../../data/lgb_feature/tags_top50.pkl')

# Gensim Doc2Vec

In [58]:
# prepare doc
def tag_doc(x):
    return x.split(',')
doc_tag = list(X['user_tags'].astype(str).apply(tag_doc).values)

In [60]:
doc_tagged = [TaggedDocument(doc, [i]) for i, doc in enumerate(doc_tag)]

In [64]:
v_size = 10
model = Doc2Vec(doc_tagged, vector_size=v_size, window=3, min_count=1, workers=16)

In [65]:
vectors_doc = []
for i in tqdm(doc_tag):
    vectors_doc.append(model.infer_vector(i))

100%|██████████| 1041674/1041674 [02:22<00:00, 7294.13it/s]


In [66]:
v_size = 10
vectors_doc = np.array(vectors_doc)
df = pd.DataFrame(data=vectors_doc,columns=['tagDim_{}'.format(i) for i in range(v_size)])
df['instance_id'] = X['instance_id'].values

In [67]:
df.to_pickle('../../data/lgb_feature/tags_gesim_window3_size10.pkl')

# DEMO

In [61]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [62]:
model.infer_vector(['human', 'interface', 'computer'])

array([-0.09725367,  0.00052088, -0.08152046,  0.03576822,  0.09515   ],
      dtype=float32)

# Analysis

In [8]:
X['user_tags'].isnull().sum()/len(X)

0.31007973703865127

In [7]:
test['user_tags'].isnull().sum()/len(test)

0.33060163901659007

In [5]:
X['user_tags'].head(20)

0                                                   NaN
1     2100191,2100078,3001825,,3001781,3001791,30017...
2                                                   NaN
3     2100098,gd_2100000,3001791,3001795,3002193,300...
4                                                   NaN
5     3002265,3002613,3002993,3003055,3003147,300331...
6                                                   NaN
7     gd_2100000,2100143,ag_2100040,3001837,3002907,...
8     ,3004484,3004430,3004434,3004490,3004468,30045...
9     2100197,2100150,2100094,2100237,2100132,ag_210...
10    2100480,2100118,2101058,2100042,2100017,210013...
11                                                  NaN
12                gd_2100001,ag_2100039,3004430,3004434
13                                                  NaN
14                                                  NaN
15    gd_2100000,3001771,3001881,3001889,3001899,300...
16                                                  NaN
17                                              

In [10]:
train_ = train[train.user_tags.notnull()].copy()
print(train_.click.sum()/len(train_))

0.19369110250332428


In [11]:

print(train.click.sum()/len(train))

0.19845954175610242


# Tag SVD

In [18]:
unique_tags = []
for each in X.user_tags.astype(str).values:
    unique_tags.extend(each.split(','))
unique_tags = list(set(unique_tags))
print(len(unique_tags))
index_dict = {}
index=0
for tag in unique_tags:
    index_dict[tag]=index
    index+=1

1406


In [19]:
from scipy.sparse import lil_matrix 
one_hot = lil_matrix ((len(X),len(index_dict)))

In [20]:
row = 0
for each in tqdm(X.user_tags.astype(str).values):
    for e in each.split(','):
        col = index_dict[e]
        one_hot[row,col] += 1
    row += 1
        

100%|██████████| 1041674/1041674 [01:50<00:00, 9459.81it/s]


In [21]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
svd.fit(one_hot)  

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=7,
       random_state=42, tol=0.0)

In [22]:
ss = svd.transform(one_hot)

In [25]:
import pickle
pickle.dump(ss,open('../../data/lgb_feature/utag_svd_50.pkl','wb'))